In [8]:
import fitz
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from PIL import Image 
import io
import boto3
import numpy as np
import re

In [2]:
import pytesseract
import cv2

# Code for Converting PDF to IMG

In [3]:

def convert_pdf2img(input_file: str,pages=None):
    """Converts pdf to image and generates a file by page"""
    # Open the document
    pdfIn = fitz.open(input_file)
    output_files = []
    # Iterate throughout the pages
    for pg in range(pdfIn.pageCount):
        if str(pages) != str(None):
            if str(pg) not in str(pages):
                continue
        # Select a page
        page = pdfIn[pg]
        rotate = int(0)
        # PDF Page is converted into a whole picture 1056*816 and then for each picture a screenshot is taken.
        # zoom = 1.33333333 -----> Image size = 1056*816
        # zoom = 2 ---> 2 * Default Resolution (text is clear, image text is hard to read)    = filesize small / Image size = 1584*1224
        # zoom = 4 ---> 4 * Default Resolution (text is clear, image text is barely readable) = filesize large
        # zoom = 8 ---> 8 * Default Resolution (text is clear, image text is readable) = filesize large
        zoom_x = 2
        zoom_y = 2
        # The zoom factor is equal to 2 in order to make text clear
        # Pre-rotate is to rotate if needed.
        mat = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
        pix = page.getPixmap(matrix=mat, alpha=False)
        path = 'Makro Invoices/Makro Invoices/Converted/'
        output_file = f"{path+os.path.splitext(os.path.basename(input_file))[0]}_page{pg+1}.png"
        pix.writePNG(output_file)
        output_files.append(output_file)
    pdfIn.close()
    summary = {
        "File": input_file, "Pages": str(pages), "Output File(s)": str(output_files)
    }
    # Printing Summary
    print("## Summary ########################################################")
    print("\n".join("{}:{}".format(i, j) for i, j in summary.items()))
    print("###################################################################")
    return output_files

# Code for Converting PDF to IMG

In [63]:
convert_pdf2img('Makro Invoices/Makro Invoices/check/03.07.20_Extintores Valle Union Sl_3243,8.pdf')

## Summary ########################################################
File:Makro Invoices/Makro Invoices/check/03.07.20_Extintores Valle Union Sl_3243,8.pdf
Pages:None
Output File(s):['Makro Invoices/Makro Invoices/Converted/03.07.20_Extintores Valle Union Sl_3243,8_page1.png']
###################################################################


['Makro Invoices/Makro Invoices/Converted/03.07.20_Extintores Valle Union Sl_3243,8_page1.png']

# Code For Extracting Fields using Tesseract 

In [5]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [84]:
%%time
dat = {
    "Vat Number":'',
       "Total":0,
       "Date":'',
      "Invoice_Number":''}
img  = cv2.imread('Makro Invoices/Makro Invoices/Converted/03.07.20_Extintores Valle Union Sl_3243,8_page1.png', cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
print(type(text_list))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):
        #vat number section
        if (re.search("[A-Z]-\d{2}/\d{6}",a)):
            b = re.findall("[A-Z]-\d{2}/\d{6}",a)
            dat["Vat Number"] = b[0]
        if (re.search("\d{8}-[A-Z]",a)):
            b = re.findall("\d{8}-[A-Z]",a)
            dat["Vat Number"] = b[0]

        if (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]              

         #Date section           
        if (re.search("\d{2}/\d{2}/\d{4}",a)):
            b  = re.findall("\d{2}/\d{2}/\d{4}", a)
            dat["Date"] = b
        if (re.search("\d{2}-\d{2}-\d{4}",a)):
            b = re.findall("\d{2}-\d{2}-\d{4}",a)
            dat["Date"] = b

        #invoice number section
        if ("Factura" in a or "ALBARAN" in a or "Factura: " in a):

            print(a)
            b = a.split(" ")
            if b[0] == "ALBARAN":
                dat["Invoice_Number"] = b[1]
            print(b)
            if len(b)>3:
                print(b)
                if len(b)>4 and b[2] !='—' and len(b[1])!=25 and len(b[1])>5:
                    dat["Invoice_Number"] = b[1]+b[2]
                    print(b)
                elif len(b)>=5 and len(b[1])==25:
                    dat["Invoice_Number"] = b[1]
                elif len(b)>6 and len(b[3])>15:
                    dat["Invoice_Number"] = b[1]+ b[2]+ b[3]
                elif len(b)>6 and b[1]== "0/0":
                    dat["Invoice_Number"] = b[1] + b[2]
                    
        if ("Tarjeta" in a or "Credito" in a or  "pagina" in a or "Total:" in a  or "TOTAL" in a):
            print("Case Total")
            b = a.split(" ")
            if len(b[-1])==2:
                
                dat["Total"] = b[-2]+b[-1]
                print(b[-2]+b[-1])
            if b[-1]=='€':
                dat["Total"] = b[-2]
            else:
                dat["Total"] = b[-1]
                print(b[-1])
        

<class 'list'>
Case Total
TOTAL
Wall time: 5.19 s


In [85]:
dat

{'Vat Number': 'B38574281',
 'Total': 'TOTAL',
 'Date': ['03/07/2020'],
 'Invoice_Number': ''}

# Code for Extracting Table 

In [59]:
im = Image.open('Makro Invoices/Makro Invoices/02.08.21_MAKRO, S.A._377,47_page2.png')

In [60]:
buffered = io.BytesIO()
im.save(buffered, format='PNG')

In [61]:
client = boto3.client('textract')
response = client.analyze_document(
    Document={'Bytes': buffered.getvalue()},
    FeatureTypes=['TABLES']
)

In [62]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

blocks = response['Blocks']
tables = map_blocks(blocks, 'TABLE')
cells = map_blocks(blocks, 'CELL')
words = map_blocks(blocks, 'WORD')
selections = map_blocks(blocks, 'SELECTION_ELEMENT')

In [63]:
def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']

In [64]:
import pandas as pd

dataframes = []

for table in tables.values():

    # Determine all the cells that belong to this table
    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

    # Determine the table's number of rows and columns
    n_rows = max(cell['RowIndex'] for cell in table_cells)
    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

    # Fill in each cell
    for cell in table_cells:
        cell_contents = [
            words[child_id]['Text']
            if child_id in words
            else selections[child_id]['SelectionStatus']
            for child_id in get_children_ids(cell)
        ]
        i = cell['RowIndex'] - 1
        j = cell['ColumnIndex'] - 1
        content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
    dataframe = pd.DataFrame(content[1:], columns=content[0])
    dataframes.append(dataframe)

In [65]:

if "MM Num. articulo" in dataframes[1] or "MM Num. artículo" in dataframes[1] or "REF." in dataframes[1] or "Num. articulo" in dataframes[1] or "Num. artículo" in dataframes[1]:
    data = pd.DataFrame(dataframes[1])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    print("Yes 1")
elif "MM Num. articulo" in dataframes[0] or "MM Num. artículo" in dataframes[0] or  "Num. articulo" in dataframes[0]:
    data = pd.DataFrame(dataframes[0])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    print("Yes 2")

elif "Bsdc SL" in dataframes[0] :
    data = pd.DataFrame(dataframes[1])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    
elif (len(dataframes)>=3 and ("MM Num. artículo" in dataframes[2] or "MM Num. artículo" in dataframes[2] 
                            or "MM Num. articulo" in dataframes[2])):
    data = pd.DataFrame(dataframes[2])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    print('Condition 2')
    
else:
    data = pd.DataFrame(dataframes[0])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    print("Yes")

Condition 2


In [66]:
data.head(100)

,MM Num. artículo,Descrip. artículo,Cont,Prec. Ud.,Cont P.,Precio,Cant.,Importe,Imp,Prec. Ud. DT,AOO,RCD
0,8414892353052,TORTILLA PATATA REDO MC 800G,RT,"2,350",3,"7,05",1,"7,05",1,,,
1,9414892322010,SALCHICHON EXTRA ARO 300G,BS,"4,290",1,"4,29",1,"4,29",1,,,
2,2062106015607,PECHUGA POLLO FILETEADA INTERF,KG,"3,890","1,560","6,07",1,"6,07",0,,,
3,2062106015300,PECHUGA POLLO FILETEADA INTERF,KG,3.890,"1,530","5,95",1,"5,95",0,,,
4,2062106015102,PECHUGA POLLO FILETEADA INTERF,KG,"3,890","1,510","5,87",1,"5,87",0,,,
5,2053941035950,TOMATE ENSALADA GG MC CJ/KG,KG,"1,890","3,595","6,79",1,"6,79",0,,,
6,8437006417761,LECHUGA BATAVIA BANDEJA,BJ,"1,190",1,"1,19",4,"4,76",0,,,
7,8427149890149,MULTIPACK SELECTA CUMBA BL 400,BL,"5,350",1,"5,35",2,"10,70",1,,,


In [67]:
data.columns

Index(['MM Num. artículo', 'Descrip. artículo', 'Cont', 'Prec. Ud.', 'Cont P.',
       'Precio', 'Cant.', 'Importe', 'Imp', 'Prec. Ud. DT', 'AOO', 'RCD'],
      dtype='object')

In [68]:

if data.iloc[0,0] == "Número de pedido" or data.iloc[0,0] =="":
    print('HI')
    big = data.columns[0]
    cont = data.columns[2]
    lot = data.columns[1]
    prec = data.columns[3]
    cant = data.columns[-5]
    imp = data.columns[-2]
    importe = data.columns[-3]
    third = data.columns[-4]
    da = data.loc[(data[big] != "Número de pedido") & (data[big] != "*** Número de pedido") & (data[big]!="*** Número de") & (data[big]!="*** Número")]
    da = da.loc[(da[big] != "Entregado a: TABERNA") & (da[big] !="Entregado a: TASCA") & (da[big]!="Entregado a: TABERNA OSUNA")
          & (da[big]!= "Entregado a: TABERNA osuna")& (da[big]!="Fin de número") ]
    da = da.loc[(da[imp]!= "Spain Fecha:") & (da[imp]!= "Spain Fecha:0") & (da[imp]!= "Spain Fecha:1")]
    da = da.loc[da[cont] !="LOT:"]
    da = da.loc[da[cant] !="Total"]
    da = da.loc[da[cant]!="LA LAGUNA" ]
    da = da.applymap(lambda x: str(x.replace(',','.'))) 
    da = da.applymap(lambda x: str(x.replace(';','.')))
    da = da.applymap(lambda x: str(x.replace(':','.')))
    da = da.replace('',np.nan)
    da = da.dropna(axis=0, thresh=3, how="any")
    limitPer = len(da) * .100
    da = da.dropna(thresh=limitPer, axis=1)
    da = da.replace(np.nan,'0')
else:
    da = data

In [69]:
data =da

In [70]:
data.columns = data.columns.str.replace('NOT_SELECTED  ', '')
data.columns = data.columns.str.replace('NOT_SELECTED', '')

In [71]:
data.columns = data.columns.str.replace('SELECTED  ', '')
data.columns = data.columns.str.replace('SELECTED ', '')
data.columns = data.columns.str.replace('SELECTED', '')
data.columns = data.columns.str.replace('NOT_', '')

In [72]:
data.columns = data.columns.str.replace(' ', '')

In [73]:
data = data.replace('','0')
data = data.applymap(lambda x: str(x.replace(',','.')))
data = data.applymap(lambda x: str(x.replace(';','.')))
#data = data.astype({'Precio':float,'Importe':float})

In [74]:
data.dtypes

MMNum.artículo      object
Descrip.artículo    object
Cont                object
Prec.Ud.            object
ContP.              object
Precio              object
Cant.               object
Importe             object
Imp                 object
Prec.Ud.DT          object
AOO                 object
RCD                 object
dtype: object

In [75]:
data.head(100)

,MMNum.artículo,Descrip.artículo,Cont,Prec.Ud.,ContP.,Precio,Cant.,Importe,Imp,Prec.Ud.DT,AOO,RCD
0,8414892353052,TORTILLA PATATA REDO MC 800G,RT,2.350,3,7.05,1,7.05,1,0,0,0
1,9414892322010,SALCHICHON EXTRA ARO 300G,BS,4.290,1,4.29,1,4.29,1,0,0,0
2,2062106015607,PECHUGA POLLO FILETEADA INTERF,KG,3.890,1.560,6.07,1,6.07,0,0,0,0
3,2062106015300,PECHUGA POLLO FILETEADA INTERF,KG,3.890,1.530,5.95,1,5.95,0,0,0,0
4,2062106015102,PECHUGA POLLO FILETEADA INTERF,KG,3.890,1.510,5.87,1,5.87,0,0,0,0
5,2053941035950,TOMATE ENSALADA GG MC CJ/KG,KG,1.890,3.595,6.79,1,6.79,0,0,0,0
6,8437006417761,LECHUGA BATAVIA BANDEJA,BJ,1.190,1,1.19,4,4.76,0,0,0,0
7,8427149890149,MULTIPACK SELECTA CUMBA BL 400,BL,5.350,1,5.35,2,10.70,1,0,0,0


In [77]:
if dat["Supplier"] == "Makro":
    
    if data.iloc[0,0] == "Número de pedido" or data.iloc[0,0] =="":
        print('HI')
        big = data.columns[0]
        cont = data.columns[2]
        lot = data.columns[1]
        prec = data.columns[3]
        cant = data.columns[-5]
        imp = data.columns[-2]
        importe = data.columns[-3]
        third = data.columns[-4]
        da = data.loc[(data[big] != "Número de pedido") & (data[big] != "*** Número de pedido") & (data[big]!="*** Número de") & (data[big]!="*** Número")]
        da = da.loc[(da[big] != "Entregado a: TABERNA") & (da[big] !="Entregado a: TASCA") & (da[big]!="Entregado a: TABERNA OSUNA")
          & (da[big]!= "Entregado a: TABERNA osuna")& (da[big]!="Fin de número") ]
        da = da.loc[(da[imp]!= "Spain Fecha:") & (da[imp]!= "Spain Fecha:0") & (da[imp]!= "Spain Fecha:1")]
        da = da.loc[da[cont] !="LOT:"]
        da = da.loc[da[cant] !="Total"]
        da = da.loc[da[cant]!="LA LAGUNA" ]
        da = da.applymap(lambda x: str(x.replace(',','.'))) 
        da = da.applymap(lambda x: str(x.replace(';','.')))
        da = da.applymap(lambda x: str(x.replace(':','.')))
        da = da.replace('',np.nan)
        da = da.dropna(axis=0, thresh=3, how="any")
        limitPer = len(da) * .100
        da = da.dropna(thresh=limitPer, axis=1)
        da = da.replace(np.nan,'0')
        da = da.astype({'Importe':float})
        one_tax = da.loc[(da['Imp'] == '1')]
        tax_one = 0
        for i in one_tax['Importe']:
            tax_one +=i
        tax_one = round(tax_one*0.03,2)
        two_tax =  da.loc[(da['Imp'] == '2')]
        tax_two = 0
        for i in two_tax['Importe']:
            tax_two +=i
        tax_two = round(tax_two*0.07,2)
        three_tax =  da.loc[(da['Imp'] == '3')]
        tax_three = 0
        for i in three_tax['Importe']:
            tax_three +=i
        tax_three = round(tax_three*0.15,2)
        final_total = 0
        for i in da['Importe']:
            final_total +=i
        final_total = round(final_total)
        final_sum = round(final_total + tax_one + tax_two+tax_three,2)
        print("Final_Sum",final_sum)
        dat['Total'] = (dat['Total'].replace(',', '.'))
        dat['Total'] = float(dat['Total'])
        dat['Total'] = int(dat['Total'])
        print(dat["Total"],"Table Sum",final_sum)
        final_sum = int(final_sum)
        if (final_sum) == dat["Total"]:
            print("Both Are equal")
    else:
        #da = data.applymap(lambda x: str(x.replace(',','.'))) 
        #da = data.applymap(lambda x: str(x.replace(';','.')))
        #da = data.applymap(lambda x: str(x.replace(':','.')))
        #da = data.replace('',np.nan)
        #limitPer = len(da) * .100
        da = data
        #da = da.dropna(thresh=limitPer, axis=1)
        da = da.astype({'Precio':float,'Importe':float})
        one_tax = da.loc[(da['Imp'] == '1')]
        tax_one = 0
        for i in one_tax['Importe']:
            tax_one +=i
        tax_one = round(tax_one*0.03,2)
        two_tax =  da.loc[(da['Imp'] == '2')]
        tax_two = 0
        for i in two_tax['Importe']:
            tax_two +=i
        tax_two = round(tax_two*0.07,2)
        three_tax =  da.loc[(da['Imp'] == '3')]
        tax_three = 0
        for i in three_tax['Importe']:
            tax_three +=i
        tax_three = round(tax_three*0.15,2)
        final_total = 0
        for i in da['Importe']:
            final_total +=i
        final_total = round(final_total)
        final_sum = round(final_total + tax_one + tax_two+tax_three,2)
        print("Final_Sum",final_sum)
        dat['Total'] = (dat['Total'].replace(',', '.'))
        if dat['Total']== 'Tarjeta':
            pass
        else:
            dat['Total'] = float(dat['Total'])
        #dat['Total'] = int(dat['Total'])
        print(dat["Total"],"Table Sum",final_sum)
        final_sum = int(final_sum)
        if (final_sum) == dat["Total"]:
            print("Both Are equal")

    
elif dat["Supplier"] =="Montesano":
    da = data
    da = da.applymap(lambda x: str(x.replace(',','.')))
    da = da.astype({'Neto':float,'Total':float})
    dat['Total'] = dat['Total'].replace(',','.')
    tot = 0
    for i in da["Total"]:
        print(i)
        tot +=i
        tot = round(tot,2)
    three_percent = da.loc[(da['IGIC'] == '3%')]
    three_var = 0 
    for i in three_percent["Total"]:
        three_var +=i
    three_var = round(three_var *0.03,2)
    tot = three_var + tot
    tot = str(tot)
    print(tot,dat["Total"])
    if tot == dat["Total"]:
        print("Both Values are equal")
    
    

Final_Sum 51.66
Tarjeta Table Sum 51.66


In [56]:
da.dtypes

MMNum.articulo       object
Descrip.artículo     object
Cont                 object
Prec.Ud.             object
ContP.               object
Precio              float64
Cant.                object
Importe             float64
Imp                  object
Prec.Ud.             object
DT                   object
AOO                  object
RCD                  object
dtype: object

In [62]:
import re
for idx,a in enumerate(text_list):
    if(a.strip() != ""):
        if (re.search("\d{1}/\d{1}(\d{3})",a)):
            b  = re.findall("\d{1}/\d{1}(\d{3})", a)
            print(b)

['021']
['021']
['474']


In [70]:
img  = cv2.imread('Makro Invoices/Makro Invoices/Converted/03.07.20_Extintores Valle Union Sl_3243,8_page1.png', cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")

In [82]:
dat = {
    "Vat Number":'',
       "Total":0,
       "Date":''}
for idx, a in enumerate(text_list):
    if(a.strip() != ""):
        #vat number section
        if (re.search("[A-Z]-\d{2}/\d{6}",a)):
            b = re.findall("[A-Z]-\d{2}/\d{6}",a)
            dat["Vat Number"] = b[0]
        if (re.search("\d{8}-[A-Z]",a)):
            b = re.findall("\d{8}-[A-Z]",a)
            dat["Vat Number"] = b[0]

        if (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]              

         #Date section           
        if (re.search("\d{2}/\d{2}/\d{4}",a)):
            b  = re.findall("\d{2}/\d{2}/\d{4}", a)
            dat["Date"] = b
        elif (re.search("\d{2}-\d{2}-\d{4}",a)):
            b = re.findall("\d{2}-\d{2}-\d{4}",a)
            dat["Date"] = b

In [83]:
dat
#s = "alpha.Customer[cus_Y4o9qMEZAugtnW] ..."

{'Vat Number': 'B38574281', 'Total': 0, 'Date': ['03/07/2020']}

In [32]:
m = re.search(r"\[(\w+)\]", s)
print(m)

<re.Match object; span=(14, 34), match='[cus_Y4o9qMEZAugtnW]'>


In [36]:
your_string = "lnfgbdgfi343456dsfidf(123) ljfbgns47647jfbgfjbgskj"
your_string[your_string.find("[")+1 : your_string.find("]")]

'lnfgbdgfi343456dsfidf(my data) ljfbgns47647jfbgfjbgsk'

In [43]:
re.findall(r"\(([a-zA-Z0-9 ._]*)\)", your_string)

[]

In [58]:
your_string = "1234lnfgbdgfi343456dsfidf(076) ljfbgns47647jfbgfjbgskj"
re.findall(r"\(([0-9{3}]*)\)", your_string)

['076']